# Instalando dependências

In [ ]:
%pip install pandas
%pip install -U google-generativeai
%pip install python-decouple
%pip install python-dotenv

%pip install sacrebleu
%pip install rouge

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Criando um promt para resumo de chunks

In [32]:
import google.generativeai as genai
from decouple import config
from dotenv import load_dotenv
from typing import List

load_dotenv()

genai.configure(api_key=config("GEMINI_KEY"))

def generate_chunks_summary(chunks: List[str]) -> str:

    formated_quotes = "\n".join([f"<quote>{quote}</quote>" for quote in chunks])

    prompt = f"""
    <summarizer>
        You are a assistant created to emulate the best summarizer model in the world. Created to summarize quotes of a television show.
    </summarizer>

    1. Allways resume the important information.
    3. Keep the summary detailed
    4. Do not include your own opinion.
    5. Do not include any new information.
    6. Do not include any examples.
    7. Do not include any quotes.
    8. Never break the main idea of the text.
    9. Those chunks are from the same episode of a television show.
    
    {formated_quotes}
    """



    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    
    return response.text

### Criando um promt para resumo de falas

In [ ]:
import google.generativeai as genai
from decouple import config
from dotenv import load_dotenv
from typing import List

load_dotenv()

genai.configure(api_key=config("GEMINI_KEY"))

def generate_quote_summary(quotes: List[str]) -> str:

    formated_quotes = "\n".join([f"<quote>{quote}</quote>" for quote in quotes])

    prompt = f"""
    <summarizer>
        You are a assistant created to emulate the best summarizer model in the world. Created to summarize quotes of a television show.
    </summarizer>

    1. Allways resume the important information.
    2. Leave out unimportant details.
    3. Keep the summary detailed
    4. Do not include your own opinion.
    5. Do not include any new information.
    6. Do not include any examples.
    7. Do not include any quotes.
    8. Never break the main idea of the text.
    
    {formated_quotes}
    """



    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    
    return response.text

In [24]:

def get_chunks(quote_list: List[str], chunk_size: int, intersection_size: int) -> List[List[str]]:
    quotes_size = len(quote_list)

    result = []

    for i in range(0, quotes_size, chunk_size):
        start = i if i - intersection_size < 0 else i - intersection_size
        end = start + chunk_size
        selected_quotes = quote_list[start:end]
        result.append(selected_quotes)

    return result

In [25]:
import pandas as pd

df = pd.read_csv("./data/thesimpsons/simpsons_script_lines.csv")
df = df.dropna(subset=["normalized_text"])
chosen_quotes = df[df["episode_id"] == 92]

chunks = get_chunks(chosen_quotes["normalized_text"].tolist(), 100, 25)

C:\Users\mateus\AppData\Local\Temp\ipykernel_14616\1308231324.py:3: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/thesimpsons/simpsons_script_lines.csv")


In [ ]:
chunks_resume = [generate_quote_summary(chunk) for chunk in chunks]

In [33]:
final_resume = generate_chunks_summary(chunks_resume)

In [38]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU()

score = bleu_scorer.sentence_score(
    hypothesis=final_resume,
    references=chunks_resume,
)

score.score/100 

It is recommended to enable `effective_order` for sentence-level BLEU.


0.16329704759666938

In [ ]:
from rouge import Rouge
rouge_scorer = Rouge()

hypothesis = "to make people trustworthy you need to trust them"
reference = "the way to make people trustworthy is to trust them"

for chunk in chunks_resume:
    score = rouge_scorer.get_scores(
        hyps=final_resume,
        refs=chunk,
    )
    print('ROUGE-L-F', score[0]["rouge-l"]["f"])
    print('ROUGE-L-R', score[0]["rouge-l"]['r'])
    print('ROUGE-L-P', score[0]["rouge-l"]['p'])
    print("=====================================")

ROUGE-L-F 0.321428566431406
ROUGE-L-R 0.32926829268292684
ROUGE-L-P 0.313953488372093
ROUGE-L-F 0.24539876802137842
ROUGE-L-R 0.2597402597402597
ROUGE-L-P 0.23255813953488372
ROUGE-L-F 0.24999999518904328
ROUGE-L-R 0.3103448275862069
ROUGE-L-P 0.20930232558139536
